## Import libraries

In [89]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm.notebook import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
np.random.seed(42)
%matplotlib inline

In [90]:
#한글 폰트 적용
from matplotlib import font_manager as fm

%matplotlib inline

f_path = 'C:/windows/Fonts/malgun.ttf'
fm.FontProperties(fname=f_path).get_name()
mpl.rc('font', family='Malgun Gothic')


## Preprocessing

In [91]:
#한글 csv 파일 읽을 때 한글 전용 인코딩'cp949'사용
#Unnamed: 0 열 drop
df_article = pd.read_csv('./datasets/df_article.csv', encoding='cp949').drop('Unnamed: 0' , axis=1)

df_article

,date,title,article,label
0,2022-05-29 23:26:00,내일 군대가네,잘 다녀오겠습니다~,[]
1,2022-05-27 23:59:00,에스파 역시나 고대 축제에서도 립싱크하던데,닝닝 고음파트에서 마이크 조정하는척 하는거 존웃,[]
2,2022-05-29 22:54:00,0,rc101이고 rc자기주도고 이젠 그냥 RC 붙은 것들 보기만 해도 발작 일어날 것...,[{'악플/욕설': 'rc101이고 rc자기주도고 이젠 그냥 RC 붙은 것들 보기만...
3,2022-05-29 21:03:00,좋은 기운 얍,이 글 쓴 사람인데 취뽀했어요\n\n꾸준히 노력하고 조급해하지 않는 게 마음처럼 쉽...,[]
4,2022-05-29 21:55:00,이때가 그리우면 개추ㅋㅋㅋㅋ,에타에서 다들 환호하던 때가 그립다,[]
...,...,...,...,...
195,2022-05-25 19:25:00,보고올게....,날 위해 기도해줘...,[]
196,2022-05-25 18:47:00,아싸 수집가 나밖에 없음?,과에서 아싸일거 같은 애한테 그냥 친해지고 싶다고 밥먹자 하고 밥먹을때 ㄹㅇ 신나게...,[]
197,2022-05-25 18:21:00,밤길 불안 자료 본 논문 팩트체크,http://repository.kihasa.re.kr/bitstream/20100...,[]
198,2022-05-25 18:27:00,아빠가 오백만년만에 영화보재서 나왔는데,무슨 영환지 끝까지 안알려주고 이제 막 잠실도착했거든\n닥스는 아닐거고 범죄도시나 ...,[{'악플/욕설': '그대가 조국 이네... 진짜 미치겠다'}]


In [92]:
#복제본 만들기
df_article_copy = df_article.copy()

In [93]:
#datetime으로 data type 변경
df_article['date'] = pd.to_datetime(df_article['date'])

#연도, 월 column 만들기
df_article['year'] = df_article['date'].dt.year
df_article['month'] = df_article['date'].dt.month

#날짜 column drop
df_article.drop('date', axis=1, inplace=True)
df_article.columns

Index(['title', 'article', 'label', 'year', 'month'], dtype='object')

In [94]:
#column 순서 변경
df_article = df_article[['year', 'month', 'title', 'article', 'label']]

df_article.head(3)

,year,month,title,article,label
0,2022,5,내일 군대가네,잘 다녀오겠습니다~,[]
1,2022,5,에스파 역시나 고대 축제에서도 립싱크하던데,닝닝 고음파트에서 마이크 조정하는척 하는거 존웃,[]
2,2022,5,0,rc101이고 rc자기주도고 이젠 그냥 RC 붙은 것들 보기만 해도 발작 일어날 것...,[{'악플/욕설': 'rc101이고 rc자기주도고 이젠 그냥 RC 붙은 것들 보기만...


In [95]:
#중복값 확인
df_article[df_article.duplicated(keep=False)]

,year,month,title,article,label
13,2022,5,0,나는 행복합니다,[]
30,2022,5,0,나는 행복합니다,[]
39,2022,5,송강호 칸 영화제 남우주연상,수상,[]
40,2022,5,송강호 칸 영화제 남우주연상,수상,[]
139,2022,5,0,양치하는데 앵무가 목욕하겠다고 세면대 자리 뺏음\n양칫물 못뱉는중,[]
140,2022,5,0,양치하는데 앵무가 목욕하겠다고 세면대 자리 뺏음\n양칫물 못뱉는중,[]
142,2022,5,0,나는 행복합니다,[]


In [96]:
#중복값 제거
df_article = df_article.drop_duplicates()

In [104]:
#혐오표현만 가져오기
df_hate = df_article[df_article['label'] != '[]']

df_hate.head(3)

,year,month,title,article,label
2,2022,5,0,rc101이고 rc자기주도고 이젠 그냥 RC 붙은 것들 보기만 해도 발작 일어날 것...,[{'악플/욕설': 'rc101이고 rc자기주도고 이젠 그냥 RC 붙은 것들 보기만...
5,2022,5,안봐서 잘 모르겠지만,정신병겜 응원하는 사람들이라 그런지 정신병이 확실하구만,[{'기타 혐오': '정신병겜 응원하는 사람들이라 그런지 정신병이 확실하구만'}]
8,2022,5,지금 신촌에서 탈원전-환경보호 시위 중인데,대체 에너지는 에너지 밀도도 충분하지 못하고 발전량도 부족하다는 것.\n이산화탄소 ...,[{'악플/욕설': '아니면 알면서 시민단체 지원금과 자기 이력서에 추가할 스펙을 ...


해당 샘플파일에는 2022년 5월 데이터만 존재해서 따로 연도, 월별로 정리하지 않음

In [115]:
#label에서 '익플/욕설' 있는 데이터 가져오기
df_hate[df_hate.label.str.contains('악플/욕설')]

,year,month,title,article,label
2,2022,5,0,rc101이고 rc자기주도고 이젠 그냥 RC 붙은 것들 보기만 해도 발작 일어날 것...,[{'악플/욕설': 'rc101이고 rc자기주도고 이젠 그냥 RC 붙은 것들 보기만...
8,2022,5,지금 신촌에서 탈원전-환경보호 시위 중인데,대체 에너지는 에너지 밀도도 충분하지 못하고 발전량도 부족하다는 것.\n이산화탄소 ...,[{'악플/욕설': '아니면 알면서 시민단체 지원금과 자기 이력서에 추가할 스펙을 ...
48,2022,5,젖닌들이 코파 수준 드립치면 웃긴게,지네 주인님 남미팀 상대전적은 알고 그러는건지 흠\n남미 출신 아닌게 다행이신듯,"[{'인종/국적': '젖닌들이 코파 수준 드립치면 웃긴게'}, {'악플/욕설': '..."
50,2022,5,0,해걸이 너무 힘들다 ㄹㅇ 슈밤바.......................,[{'악플/욕설': '해걸이 너무 힘들다 ㄹㅇ 슈밤바....................
56,2022,5,'그 손가락' 논란 피해감,아니 근데 시발,[{'악플/욕설': '아니 근데 시발'}]
57,2022,5,동민게이야….,부끄러운 줄 알아야지,[{'악플/욕설': '부끄러운 줄 알아야지'}]
59,2022,5,0,제발 정신 차려라…. 뭐하는 놈인지 모르겠는데 만나자는 글에 들어갔다가 너무 무례하...,[{'악플/욕설': '제발 정신 차려라…. 뭐하는 놈인지 모르겠는데 만나자는 글에 ...
73,2022,5,한국사회 약간 텅빈껍데기같음,"사회전부가 외모, 숫자에 집착하는데 그것도 개도국일때나 쓸모있지\n\n뭐랄까 철학적...","[{'악플/욕설': '한국사회 약간 텅빈껍데기같음'}, {'여성/가족': '사회전부..."
84,2022,5,확실히 연예인 공연 자체는 앜랔>>입실이다,라인업 떠나서 고대는 너무 넓고 사람도 많아서 분위기가 너무 어수선해서 첨에 놀랐음...,[{'악플/욕설': '아카라카는 아가리들이 추임새 넣어주면서 일어나게 하고 호응 유...
87,2022,5,0,하루종이 똥같은 코드만 싸고있네,[{'악플/욕설': '하루종이 똥같은 코드만 싸고있네'}]
